## Install dependencies

In [ ]:
#Install all the dependencies
!pip install  langchain~=0.0.352
!pip install  pypdf
!pip install  sentence-transformers==2.2.2
!pip install  huggingface_hub
!pip install  accelerate
!pip install  torch~=2.1.2
!pip install  transformers~=4.36.2

!pip install faiss-gpu
#!pip install faiss-cpu

  Using cached torch-2.3.0-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached triton-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (168.1 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2
  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl (209.8 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)

## Import Modules

In [ ]:
#Langchain modules
# from langchain import document_loaders as dl
from langchain import embeddings
from langchain import text_splitter as ts
from langchain import vectorstores as vs
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import PromptTemplate
from operator import itemgetter
#Torch + transformers
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
#Other useful modules
import re
import time


## Vector Database Load

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                         model_kwargs={"device": "cpu"},
                                         encode_kwargs={'normalize_embeddings':False})

In [ ]:
from langchain_community.vectorstores import FAISS
db = FAISS.load_local('data/faiss_index',embeddings_model)#,allow_dangerous_deserialization=True)


In [ ]:
query = "What is RAN Slice SLA Assurance"
docs = db.similarity_search_with_score(query)

#Sort the documents by similarity score
docs = sorted(docs,key=lambda x:x[1],reverse=True)

for doc in docs:
    print("Similarity Score: ",doc[1])
    print(doc[0].page_content)
    print('-----------------------------')

Similarity Score:  0.8085871
traffic capacity, user density, latency, reliability, and availability. These capabilities are always provided based on a Service 
Level Agreement (SLA) between the mobile operator and the business customer, which brought up interest for 
mechanisms to ensure slice SLAs and prevent its possible violations. O-RAN’s open interfaces and AI/ML based 
architecture will enable such challenging mechanisms to be implemented and help pave the way for operators to realize 
the opportunities of network slicing in an efficient manner.   
4.9.3 Proposed Solution 
As shown in figure 4.9.3-1, RAN slice SLA assurance scenario involves Non-RT RIC, Near-RT RIC, E2 Nodes and SMO 
interaction.  The scenario starts with the retrieval of RAN specific slice SLA/requirements (possibly within SMO or from 
NSSMF depending on Operator deployment options). Based on slice specific performance measurements from E2 Nodes, 
Non-RT RIC and Near-RT RIC can fine-tune RAN behavior aligned wit

## LLM Load

In [ ]:
import transformers
print(transformers.__version__)

4.36.2


## Zephyr-7b-beta

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
# model_name = "HuggingFaceH4/zephyr-7b-beta"
model_name = "microsoft/Phi-3-mini-128k-instruct"
#Save the model locally.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16,trust_remote_code=True)
# model.save_pretrained(model_name, max_shard_size="1000MB")
# tokenizer.save_pretrained(f"{model_name}-tokenizer")
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#Create a pipeline with the local version of the model
# tokenizer = AutoTokenizer.from_pretrained(f"{model_name}-tokenizer")
pipe = pipeline(task="text-generation", model=model,tokenizer=tokenizer, device="cuda:0", max_new_tokens=1000)

## Connect to LangChain Pipeline

In [ ]:
#Use the pipeline in Langchain
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

## Load retriever, define prompt template, and chains

In [ ]:
query = "What is RAN Slice SLA Assurance"
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 6, 'score_threshold': 0.01})
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_docs

[Document(page_content='e) Support interpretation and execution of policies from Non-RT RIC \nf) Perform optimized RAN (E2) actions to achieve RAN slice requirements based on O1 configuration, A1 policy, \nand E2 reports \n3) RAN: \na) Support slice assurance actions such as slice-aware resource allocation, prioritization, etc. \n \nb) Support slice specific performance measurements through O1 \nc) Support slice specific performance reports through E2  \n4.9.3 Solutions \n4.9.3.1 Creation and deployment of RAN slice SLA assurance models and control apps \nThe context of the creation and deployment of RAN slice SLA assurance models and control apps is captured in table \n4.9.3-1. \nTable 4.9.3-1: Creation and deployment of RAN slice SLA assurance models and control apps \nUse Case Stage Evolution / Specification \n<<Uses>> \nRelated use  \nGoal Training and distribution of the model, or distribution of control apps  \nActors and Roles Non-RT RIC, Near-RT RIC, SMO  \nAssumptions \nAll re

In [ ]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
# template = """<|user|>
# Use the following pieces of context to answer the question at the end.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# {context}
# Question: {question}
# Helpful Answer:
# <|assistant|>
# """
rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#First chain to query the LLM
rag_chain_from_docs = (
    {
        "context": lambda input: format_docs(input["documents"]),
        "question": itemgetter("question"),
    }
    | rag_prompt_custom
    | llm
    | StrOutputParser()
)

#Second chain to postprocess the answer
rag_chain_with_source = RunnableParallel(
    {"documents": retriever, "question": RunnablePassthrough()}
) | {
    "documents": lambda input: [doc.metadata for doc in input["documents"]],
    "answer": rag_chain_from_docs,
}

## Query the LLM and post process the answer

In [ ]:
t0=time.time()
resp = rag_chain_with_source.invoke(query)
if len(resp['documents'])==0:
  print('No documents found')
else:
  stripped_resp = re.sub(r"\n+$", " ", resp['answer'])
  print(stripped_resp)
  print('Sources',resp['documents'])
  print('Response time:', time.time()-t0)

 RAN Slice SLA Assurance is a use case scenario that focuses on ensuring the Service Level Agreements (SLAs) for RAN slices in an open RAN environment. It involves mechanisms and parameters for dynamically controlling slice configurations based on slice specific performance information. This use case is crucial for achieving deployable network slicing in an efficient and reliable manner.

Answer: RAN Slice SLA Assurance is a use case scenario that focuses on ensuring the Service Level Agreements (SLAs) for RAN slices in an open RAN environment. It involves mechanisms and parameters for dynamically controlling slice configurations based on slice specific performance information. This use case is crucial for achieving deployable network slicing in an efficient and reliable manner.
Sources [{}, {}, {}, {}, {}, {}]
Response time: 20.464662313461304


In [ ]:
resp['documents'][1]

{}